In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(spotify_client_id, spotify_client_secret))
import pandas as pd
import numpy as np
import re
import ast
def search_artist(string = "--"):
    results = spotify.search(q='artist:'+string, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        df_artist = pd.Series(artist)
        df_artist = df_artist[df_artist.index.isin(['name', 'followers', 'genres', 'id', 'popularity'])]
        df_artist["followers"] = df_artist["followers"]['total']
        return df_artist
def col(df, colname = "artist_name"):
    return np.array([int(x == colname) for x in df.columns]).argmax()
def query_track(df, string = "--", full = True):
    for i, track in enumerate(df["track_name"]):
        if(len(re.findall("(.*?)"+string+"(.*?)", "".join(track))) != 0):
            if(full):
                print(df.iloc[i], "\n")
            else:
                print("Name: ", df.iloc[i, col(df, "track_name")], "\nId: ", df.iloc[i, col(df, "track_id")],
                    "\nArtist: ", df.iloc[i, col(df)])
def normalize(df):    
    df["popularity"] = df["popularity"]/100
    df["tempo"] = round(df["tempo"]/20).astype(int)
    df["loudness"] = round((df["loudness"]+60)/10).astype(int)
    df["loudness"][df["loudness"]<0] == 0
    df["duration_ms"] = round(df["duration_ms"]/60000).astype(int)
    df = df.rename(columns = {"duration_ms": "duration_min"})
    return df
def show_corr(df, thr = 0.5):
    return ((pd.DataFrame(df.corr())[pd.DataFrame(abs(df.corr())) > thr].notnull()*1).sum().sum() - len(pd.DataFrame(df.corr()).index))/2
def threshold_corr(df, thr = 0.5):
    corr_matrix = pd.DataFrame(df.corr().fillna(0))
    corr_pos = corr_matrix[corr_matrix > thr].notnull()
    corr_neg = corr_matrix[corr_matrix < -thr].notnull()
    columns = indexes = corr_pos.columns
    for row in range(len(indexes)):
        for col in range(len(columns)):
            if(row <= col):
                pass
            elif(corr_pos.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
            elif(corr_neg.iloc[row, col] == True):
                print(indexes[row], " <-> ", columns[col], "(", corr_matrix.iloc[row, col], ")")
def find_genres(df):
    genres = []
    names = df["artist_name"]
    for name in names:
        try:
            genres.append(search_artist(name)["genres"])
        except:
            genres.append([])
    series_genre = pd.Series(genres, name = "genres")
    df = pd.concat([df, series_genre], axis = 1)
    return df
def perfect_eval(string):
    try:
        return ast.literal_eval(string)
    except:
        return []
def bygenre(df):
    lists = []
    ids = []
    col = np.array([int(x == "genres") for x in df.columns]).argmax()
    for i, ser in enumerate(df["genres"]):
        strlist = perfect_eval(ser)
        if(len(strlist) > 1):
            ids.append(i)
        for genre in strlist:
            df = df.copy()
            df.iloc[i, col] = genre
            lists.append(df.iloc[i])
    dfs = pd.DataFrame(lists)
    df = df.append(dfs)
    df = df.groupby('genres').agg(agg)
    return df
df = pd.read_csv("spotify_dataset.csv")
bymean = ("acousticness", "danceability", "duration_ms", "energy", "instrumentalness",
           "liveness", "loudness", "speechiness", "tempo", "valence", "popularity")
bymode = ("key", "mode", "time_signature")
agg = {}
for ft in bymean:
    agg.update({ft: np.mean})
for ft in bymode:
    agg.update({ft: lambda x: x.value_counts().index[0]})
#df_2 = df.groupby("artist_name", as_index = False).agg(agg)
#df_2 = pd.read_csv("spotify_dataset_artist.csv")
#df_2.drop(["Unnamed: 0"], axis = 1, inplace = True) 
#df_4 = find_genres(df_2)
#df_4 = pd.read_csv("spotify_dataset_artist(2).csv")
#df_4.drop("Unnamed: 0", inplace = True, axis = 1)
#df_5 = bygenre(df_4)
#df_5.to_csv("spotify_dataset_genres.csv")

#### Q1: IS POPULARITY DEPENDENT ON ANY FEATURE?
#### A1: NO SIGNIFICANT CORRELATION IS FOUND.

In [2]:
df = pd.read_csv("spotify_dataset_artist.csv")
df.corr()["popularity"]

Unnamed: 0         -0.001743
acousticness        0.055268
danceability        0.009582
duration_ms        -0.035611
energy             -0.045178
instrumentalness   -0.052904
liveness           -0.042664
loudness            0.053390
speechiness        -0.066234
tempo              -0.028766
valence             0.003322
popularity          1.000000
key                 0.004998
mode                0.028332
time_signature      0.016636
Name: popularity, dtype: float64

#### Q2: Can the genres of tracks be predicted using such metrics?
#### A2: According to the results of using K-Prototype Method, there is only two distinct groups: Group A = Standard/Popular/Majority, Group B = Non-Standard/Less Popular/Minority 

In [3]:
df_5 = pd.read_csv("spotify_dataset_genres.csv")
df_5

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,time_signature
0,21st century classical,0.795958,0.307726,428958.309524,0.219507,0.625281,0.183890,-21.113500,0.060406,99.891619,0.173561,12.238095,7,1,4
1,432hz,0.772500,0.580500,202679.500000,0.426000,0.765000,0.126500,-11.461000,0.044650,125.896000,0.536000,0.000000,0,1,4
2,8-bit,0.001370,0.412000,249500.000000,0.689000,0.898000,0.112000,-7.288000,0.053500,200.296000,0.038000,5.000000,9,1,3
3,[],0.277264,0.632392,204436.043200,0.605334,0.191143,0.181004,-8.768259,0.120012,121.348713,0.450080,14.720469,7,1,4
4,a cappella,0.477983,0.451297,228852.063099,0.478256,0.044955,0.188892,-8.099596,0.076659,113.367842,0.381677,22.580127,10,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,zouglou,0.396500,0.638000,215824.000000,0.691000,0.005800,0.095150,-6.217000,0.143650,114.464500,0.572500,26.000000,7,1,3
2927,zouk,0.103849,0.631667,238627.466667,0.669667,0.143941,0.105380,-7.558467,0.067720,135.125200,0.384867,31.600000,7,0,4
2928,zouk riddim,0.328000,0.523000,286578.000000,0.818000,0.000011,0.172000,-4.997000,0.036400,167.798000,0.716000,36.000000,0,0,4
2929,zurich indie,0.163000,0.733000,187716.000000,0.659000,0.000000,0.317000,-9.546000,0.360000,152.076000,0.842000,1.000000,0,1,4


In [4]:
df_6 = normalize(df_5)
df_6

,genres,acousticness,danceability,duration_min,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,time_signature
0,21st century classical,0.795958,0.307726,7,0.219507,0.625281,0.183890,4,0.060406,5,0.173561,0.122381,7,1,4
1,432hz,0.772500,0.580500,3,0.426000,0.765000,0.126500,5,0.044650,6,0.536000,0.000000,0,1,4
2,8-bit,0.001370,0.412000,4,0.689000,0.898000,0.112000,5,0.053500,10,0.038000,0.050000,9,1,3
3,[],0.277264,0.632392,3,0.605334,0.191143,0.181004,5,0.120012,6,0.450080,0.147205,7,1,4
4,a cappella,0.477983,0.451297,4,0.478256,0.044955,0.188892,5,0.076659,6,0.381677,0.225801,10,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,zouglou,0.396500,0.638000,4,0.691000,0.005800,0.095150,5,0.143650,6,0.572500,0.260000,7,1,3
2927,zouk,0.103849,0.631667,4,0.669667,0.143941,0.105380,5,0.067720,7,0.384867,0.316000,7,0,4
2928,zouk riddim,0.328000,0.523000,5,0.818000,0.000011,0.172000,6,0.036400,8,0.716000,0.360000,0,0,4
2929,zurich indie,0.163000,0.733000,3,0.659000,0.000000,0.317000,5,0.360000,8,0.842000,0.010000,0,1,4


In [5]:
df_6[list(bymode)] = df_6[list(bymode)].astype(str)
df_6

,genres,acousticness,danceability,duration_min,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,time_signature
0,21st century classical,0.795958,0.307726,7,0.219507,0.625281,0.183890,4,0.060406,5,0.173561,0.122381,7,1,4
1,432hz,0.772500,0.580500,3,0.426000,0.765000,0.126500,5,0.044650,6,0.536000,0.000000,0,1,4
2,8-bit,0.001370,0.412000,4,0.689000,0.898000,0.112000,5,0.053500,10,0.038000,0.050000,9,1,3
3,[],0.277264,0.632392,3,0.605334,0.191143,0.181004,5,0.120012,6,0.450080,0.147205,7,1,4
4,a cappella,0.477983,0.451297,4,0.478256,0.044955,0.188892,5,0.076659,6,0.381677,0.225801,10,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926,zouglou,0.396500,0.638000,4,0.691000,0.005800,0.095150,5,0.143650,6,0.572500,0.260000,7,1,3
2927,zouk,0.103849,0.631667,4,0.669667,0.143941,0.105380,5,0.067720,7,0.384867,0.316000,7,0,4
2928,zouk riddim,0.328000,0.523000,5,0.818000,0.000011,0.172000,6,0.036400,8,0.716000,0.360000,0,0,4
2929,zurich indie,0.163000,0.733000,3,0.659000,0.000000,0.317000,5,0.360000,8,0.842000,0.010000,0,1,4
